# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import re

import nltk
nltk.download(['punkt', 'wordnet'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer


from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble  import BaggingClassifier
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [3]:
# load data from database
engine = create_engine('sqlite:///categorias_base.db')
df = pd.read_sql('SELECT * FROM categorias_base', engine)
X = df.message.values
Y = df.iloc[:, 4:39]

In [4]:
Y.describe()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather
count,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.0,...,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000
mean,0.773650,0.170659,0.004501,0.414251,0.079493,0.050084,0.027617,0.017966,0.032804,0.0,...,0.004577,0.011787,0.043904,0.278341,0.082202,0.093187,0.010757,0.093645,0.020217,0.052487
std,0.435276,0.376218,0.066940,0.492602,0.270513,0.218122,0.163875,0.132831,0.178128,0.0,...,0.067502,0.107927,0.204887,0.448191,0.274677,0.290700,0.103158,0.291340,0.140743,0.223011
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [5]:
Y['related'].replace(2, 1, inplace=True)

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    
    text = re.sub(r'[^\w\s]','',text)
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [57]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [58]:
y_pred = pipeline.predict(X_test)
y_pred_df = pd.DataFrame(y_pred, columns=y_test.columns)

evaluation = {}
for column in y_test.columns:
    evaluation[column] = []
    evaluation[column].append(precision_score(y_test[column], y_pred_df[column]))
    evaluation[column].append(recall_score(y_test[column], y_pred_df[column]))
    evaluation[column].append(f1_score(y_test[column], y_pred_df[column]))
    
pd.DataFrame(evaluation)

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather
0,0.827062,0.824701,0.0,0.742236,0.571429,0.580645,0.444444,0.0,0.833333,0.0,...,0.0,0.0,0.125000,0.838793,0.891473,0.735507,0.0,0.884701,0.789474,0.416667
1,0.926762,0.360313,0.0,0.527400,0.052731,0.055046,0.045455,0.0,0.024752,0.0,...,0.0,0.0,0.004032,0.540255,0.223301,0.347009,0.0,0.653028,0.094937,0.014493
2,0.874078,0.501514,0.0,0.616642,0.096552,0.100559,0.082474,0.0,0.048077,0.0,...,0.0,0.0,0.007812,0.657210,0.357143,0.471545,0.0,0.751412,0.169492,0.028011


### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
parameters = parameters = {
        'vect__ngram_range': ((1, 1), (1, 2)),
        'vect__max_df': (0.75, 1.0),
        'tfidf__use_idf': (True, False)
    }

cv = GridSearchCV(pipeline, param_grid=parameters)

cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'vect__ngram_range': ((1, 1), (1, 2)), 'vect__max_df': (0.75, 1.0), 'tfidf__use_idf': (True, False)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [12]:
y_pred = cv.predict(X_test)

In [13]:
print("Accuracy")
print((y_pred == y_test).mean())

Accuracy
related                   0.804242
request                   0.877327
offer                     0.997101
aid_related               0.724596
medical_help              0.925236
medical_products          0.949496
search_and_rescue         0.973299
security                  0.981691
military                  0.969332
child_alone               1.000000
water                     0.952853
food                      0.926457
shelter                   0.926457
clothing                  0.986573
money                     0.979554
missing_people            0.989625
refugees                  0.966433
death                     0.960024
other_aid                 0.870156
infrastructure_related    0.938969
transport                 0.955905
buildings                 0.951175
electricity               0.978181
tools                     0.994660
hospitals                 0.990998
shops                     0.996491
aid_centers               0.990082
other_infrastructure      0.956057
weather_rel

In [14]:
y_pred_df = pd.DataFrame(y_pred, columns=y_test.columns)

evaluation = {}
for column in y_test.columns:
    evaluation[column] = []
    evaluation[column].append(precision_score(y_test[column], y_pred_df[column]))
    evaluation[column].append(recall_score(y_test[column], y_pred_df[column]))
    evaluation[column].append(f1_score(y_test[column], y_pred_df[column]))
    
pd.DataFrame(evaluation)


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather
0,0.832505,0.815789,0.0,0.758545,0.613636,0.710526,0.473684,0.0,0.722222,0.0,...,0.0,0.0,0.333333,0.833333,0.897727,0.761905,0.600000,0.927500,0.705882,0.652174
1,0.932618,0.380702,0.0,0.501641,0.054000,0.077810,0.051724,0.0,0.062201,0.0,...,0.0,0.0,0.003484,0.453039,0.295327,0.303448,0.047619,0.598387,0.085714,0.042857
2,0.879723,0.519139,0.0,0.603906,0.099265,0.140260,0.093264,0.0,0.114537,0.0,...,0.0,0.0,0.006897,0.586972,0.444444,0.434032,0.088235,0.727451,0.152866,0.080429


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(BaggingClassifier()))
    ])

pipeline.fit(X_train, y_train)

In [11]:
y_pred = pipeline.predict(X_test)
print("Accuracy")
print((y_pred == y_test).mean())

Accuracy
related                   0.803021
request                   0.895484
offer                     0.994202
aid_related               0.757705
medical_help              0.928288
medical_products          0.959109
search_and_rescue         0.975587
security                  0.981538
military                  0.972078
child_alone               1.000000
water                     0.962008
food                      0.946292
shelter                   0.942325
clothing                  0.989167
money                     0.979249
missing_people            0.988557
refugees                  0.969484
death                     0.965975
other_aid                 0.870613
infrastructure_related    0.936832
transport                 0.960482
buildings                 0.962313
electricity               0.983216
tools                     0.993592
hospitals                 0.986115
shops                     0.994965
aid_centers               0.987336
other_infrastructure      0.955905
weather_rel

In [12]:
y_pred_df = pd.DataFrame(y_pred, columns=y_test.columns)

evaluation = {}
for column in y_test.columns:
    evaluation[column] = []
    evaluation[column].append(precision_score(y_test[column], y_pred_df[column]))
    evaluation[column].append(recall_score(y_test[column], y_pred_df[column]))
    evaluation[column].append(f1_score(y_test[column], y_pred_df[column]))
    
pd.DataFrame(evaluation)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather
0,0.820668,0.762155,0.142857,0.756279,0.559829,0.600000,0.603774,0.130435,0.577586,0.0,...,0.0,0.173913,0.376471,0.841304,0.862857,0.788698,0.529412,0.874286,0.579710,0.483146
1,0.948959,0.535055,0.030303,0.604461,0.263052,0.337748,0.187135,0.028846,0.333333,0.0,...,0.0,0.058824,0.119403,0.648241,0.559259,0.533223,0.257143,0.765000,0.279720,0.130699
2,0.880163,0.628726,0.050000,0.671901,0.357923,0.432203,0.285714,0.047244,0.422713,0.0,...,0.0,0.087912,0.181303,0.732261,0.678652,0.636274,0.346154,0.816000,0.377358,0.205742


### 9. Export your model as a pickle file

In [ ]:
filename = 'finalized_model.pkl'
pickle.dump(pipeline, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.